In [ ]:
# CM2015 Programming with Data - Midterm Coursework
# Midterm Project: MediBot - Healthcare Assistant

## 1. Project Overview

**Chatbot Title:** MediBot - Healthcare Assistant Chatbot
**Domain:** Healthcare Information and Basic Medical Guidance
**Purpose:** Help people with basic health questions and booking appointments

## 2. Import Libraries

# Libraries needed for the chatbot
import json
import re
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

# Download NLTK data files
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
except:
    pass

## 3. Chatbot Class Definition

class MediBot:
    def __init__(self, intents_file='intents.json'):
        """
        Set up the chatbot with data from JSON file
        """
        self.intents_file = intents_file
        self.intents = {}
        self.pattern2intent = {}  # maps patterns to intents
        self.intent2response = {}  # maps intents to responses
        self.user_memory = {}  # stores user info like name, age
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        
        # Load data and build dictionaries
        self.load_intents()
        self.build_pattern_dictionaries()
    
    def load_intents(self):
        """
        Load intents data from JSON file
        """
        try:
            with open(self.intents_file, 'r', encoding='utf-8') as file:
                data = json.load(file)
                self.intents = data['intents']
                print(f"Loaded {len(self.intents)} intents from {self.intents_file}")
        except FileNotFoundError:
            print(f"Error: {self.intents_file} not found!")
            self.intents = []
        except json.JSONDecodeError:
            print(f"Error: Bad JSON format in {self.intents_file}")
            self.intents = []
    
    def build_pattern_dictionaries(self):
        """
        Build dictionaries to map patterns to intents and intents to responses
        """
        for intent in self.intents:
            intent_tag = intent['tag']
            
            # Map each pattern to its intent
            for pattern in intent['patterns']:
                self.pattern2intent[pattern] = intent_tag
            
            # Map intent to its responses
            self.intent2response[intent_tag] = intent['responses']
    
    def preprocess_text(self, text):
        """
        Clean up user input text
        - Make lowercase
        - Remove punctuation
        - Split into words
        - Remove common words like 'the', 'a'
        - Reduce words to root form
        """
        # Make lowercase
        text = text.lower()
        
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # Split into words
        tokens = word_tokenize(text)
        
        # Remove common words and reduce to root form
        processed_tokens = []
        for token in tokens:
            if token not in self.stop_words:
                stemmed_token = self.stemmer.stem(token)
                processed_tokens.append(stemmed_token)
        
        return ' '.join(processed_tokens), tokens
    
    def extract_entities(self, user_input):
        """
        Find important info in user input like names, ages, phone numbers
        """
        entities = {}
        
        # Find name patterns
        name_pattern = r'(?:my name is|i am|i\'m|call me)\s+([a-zA-Z]+)'
        name_match = re.search(name_pattern, user_input.lower())
        if name_match:
            entities['name'] = name_match.group(1).capitalize()
        
        # Find age patterns
        age_pattern = r'(?:i am|i\'m)\s+(\d+)\s+years?\s+old|age\s+(\d+)|(\d+)\s+years?\s+old'
        age_match = re.search(age_pattern, user_input.lower())
        if age_match:
            age = age_match.group(1) or age_match.group(2) or age_match.group(3)
            entities['age'] = age
        
        # Find phone number patterns
        phone_pattern = r'(\d{3}[-.\s]?\d{3}[-.\s]?\d{4}|\d{10})'
        phone_match = re.search(phone_pattern, user_input)
        if phone_match:
            entities['phone'] = phone_match.group(1)
        
        return entities
    
    def pattern_matching(self, user_input):
        """
        Check if user input matches any of our patterns
        """
        processed_input, tokens = self.preprocess_text(user_input)
        
        # Try each pattern to see if it matches
        for pattern, intent_tag in self.pattern2intent.items():
            # Create regex pattern that ignores case
            regex_pattern = re.compile(pattern, re.IGNORECASE)
            
            # Check original input
            if regex_pattern.search(user_input):
                return intent_tag
            
            # Check processed input
            if regex_pattern.search(processed_input):
                return intent_tag
        
        return None
    
    def generate_response(self, intent_tag, user_input):
        """
        Create a response based on the intent and user input
        """
        if intent_tag not in self.intent2response:
            return "Sorry, I didn't understand that. Can you try again?"
        
        # Get possible responses for this intent
        responses = self.intent2response[intent_tag]
        
        # Pick a random response
        response = random.choice(responses)
        
        # Find and save any important info from user input
        entities = self.extract_entities(user_input)
        self.user_memory.update(entities)
        
        # Replace placeholders in response with user info
        response = self.substitute_placeholders(response)
        
        return response
    
    def substitute_placeholders(self, response):
        """
        Replace {name}, {age}, {phone} in responses with actual user info
        """
        if '{name}' in response and 'name' in self.user_memory:
            response = response.replace('{name}', self.user_memory['name'])
        
        if '{age}' in response and 'age' in self.user_memory:
            response = response.replace('{age}', self.user_memory['age'])
        
        if '{phone}' in response and 'phone' in self.user_memory:
            response = response.replace('{phone}', self.user_memory['phone'])
        
        return response
    
    def chat(self):
        """
        Main chat loop - keeps running until user says exit
        """
        print("=" * 60)
        print("🏥 Welcome to MediBot - Healthcare Assistant Chatbot 🏥")
        print("=" * 60)
        print("I can help with health questions and booking appointments.")
        print("Type 'exit' or 'quit' to stop chatting.")
        print("-" * 60)
        
        while True:
            user_input = input("\nYou: ").strip()
            
            # Check if user wants to exit
            if user_input.lower() in ['exit', 'quit', 'bye', 'goodbye']:
                print("\nMediBot: Thanks for using MediBot! Stay healthy! 👋")
                break
            
            if not user_input:
                print("MediBot: Please type something.")
                continue
            
            # Find what the user wants
            intent = self.pattern_matching(user_input)
            
            if intent:
                response = self.generate_response(intent, user_input)
                print(f"MediBot: {response}")
            else:
                print("MediBot: Sorry, I didn't understand. Can you ask differently?")

## 4. Test Cases and Demonstration

def run_test_cases():
    """
    Test the chatbot with different inputs
    """
    print("=" * 60)
    print("🧪 TESTING THE CHATBOT 🧪")
    print("=" * 60)
    
    bot = MediBot()
    
    test_cases = [
        {
            "name": "Test 1: Saying Hello and Giving Info",
            "inputs": [
                "Hello",
                "My name is Alice",
                "I am 25 years old"
            ],
            "description": "Tests if bot remembers my name and age"
        },
        {
            "name": "Test 2: Asking About Health Problems",
            "inputs": [
                "I have a headache",
                "What should I do for fever?",
                "I'm feeling nauseous"
            ],
            "description": "Tests if bot gives good health advice"
        },
        {
            "name": "Test 3: Booking Appointments",
            "inputs": [
                "I want to book an appointment",
                "Schedule a consultation",
                "My phone number is 555-123-4567"
            ],
            "description": "Tests appointment booking and saving phone numbers"
        }
    ]
    
    for test_case in test_cases:
        print(f"\n{test_case['name']}")
        print(f"What this tests: {test_case['description']}")
        print("-" * 40)
        
        for user_input in test_case['inputs']:
            intent = bot.pattern_matching(user_input)
            if intent:
                response = bot.generate_response(intent, user_input)
                print(f"User says: {user_input}")
                print(f"Bot thinks: {intent}")
                print(f"Bot replies: {response}")
            else:
                print(f"User says: {user_input}")
                print("Bot thinks: Don't understand")
                print("Bot replies: Sorry, I didn't understand.")
            print()
    
    print("What bot remembers about user:", bot.user_memory)

## 5. Advanced Features Demonstration

def demonstrate_advanced_features():
    """
    Show off the cool NLP features
    """
    print("=" * 60)
    print("🚀 COOL FEATURES DEMO 🚀")
    print("=" * 60)
    
    bot = MediBot()
    
    # Show text cleaning
    sample_text = "Hello! I'm feeling really bad today, and I have a terrible headache."
    processed, tokens = bot.preprocess_text(sample_text)
    
    print("Text Cleaning:")
    print(f"Before: {sample_text}")
    print(f"After: {processed}")
    print(f"Words: {tokens}")
    
    # Show finding names/ages/phones
    sample_input = "Hi, my name is John and I am 30 years old. My phone is 555-987-6543."
    entities = bot.extract_entities(sample_input)
    
    print(f"\nFinding Info in Text:")
    print(f"User said: {sample_input}")
    print(f"Found: {entities}")
    
    # Show pattern matching
    print(f"\nPattern Matching Test:")
    test_inputs = [
        "I have a severe headache",
        "My head hurts badly", 
        "I'm experiencing head pain"
    ]
    
    for test_input in test_inputs:
        intent = bot.pattern_matching(test_input)
        print(f"'{test_input}' -> {intent}")

## 6. Running Everything

if __name__ == "__main__":
    # Run tests first
    run_test_cases()
    
    # Show cool features
    demonstrate_advanced_features()
    
    # Start chatting
    print("\n" + "=" * 60)
    print("🎯 CHAT WITH THE BOT 🎯")
    print("=" * 60)
    
    # Start the chatbot
    medibot = MediBot()
    medibot.chat()